In [1]:
#install dependencies
#pip install pandas xlrd sqlalchemy 

import sqlite3
import pandas as pd
import os
from sqlalchemy import create_engine
import io
from zipfile import ZipFile
import urllib
import zipfile



In [2]:
def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn:
        return zf.read(extract_fn)
    else:
        return {name:zf.read(name) for name in zf.namelist()}

In [6]:
Dir_path = 'P:/OE/OE5-DEMO/Branch1/Staff Folders/EQR_Data_Intern/test'

r = Dir_path.replace("/", "\\")

#EIA20 = 'P:\OE\OE5-DEMO\Branch1\Staff Folders\EQR_Data_Intern\test\EIA20'
EIA20r = r'P:\OE\OE5-DEMO\Branch1\Staff Folders\EQR_Data_Intern\test\EIA20'



In [8]:
EIA20r

'P:\\OE\\OE5-DEMO\\Branch1\\Staff Folders\\EQR_Data_Intern\\test\\EIA20'

In [12]:
EIA20r = r + '\\EIA20'

In [4]:
os.chdir(Dir_path)

In [13]:
EIA20r

'P:\\OE\\OE5-DEMO\\Branch1\\Staff Folders\\EQR_Data_Intern\\test\\EIA20'

In [19]:
Current_year = '20'
e_Years = ['15','16','17','18','19']

e_Years.append(Current_year)

In [9]:
input_string = input("Enter family members separated by comma ")
family_list  = input_string.split(",")
family_list

Enter family members separated by comma 15,12,16


['15', '12', '16']

In [6]:
Current_year = float(input("What is the current year? --NOTE: list the last two digits ex. 2020 enter 20 --")) 

What is the current year? --NOTE: list the last two digits ex. 2020 enter 20 --20


In [7]:
e_Years = []
e_Years = float(input("What is the current year? --NOTE: list the last two digits ex. 2020 enter 20 --")) 

What is the current year? --NOTE: list the last two digits ex. 2020 enter 20 --15, 16, 17


ValueError: could not convert string to float: '15, 16, 17'

In [3]:
e_Years

'15, 16, 17'

In [4]:
e_Years.append(Current_year)

AttributeError: 'str' object has no attribute 'append'

In [16]:
Current_year

'20'

In [ ]:
#download zip files from EIA website and in directory path folder:
for i in range(len(e_Years)):
    if (i==len(e_Years)-1):
        urlc = "https://www.eia.gov/electricity/data/eia861/zip/f86120" + Current_year + '.zip'
        E861c = urllib.request.urlretrieve(urlc, "EIA20" + Current_year + ".zip")
    else:
        url = 'https://www.eia.gov/electricity/data/eia861/archive/zip/f86120' + e_Years[i] + '.zip'
        E861 = urllib.request.urlretrieve(url, "EIA20" + e_Years[i] + ".zip")

In [26]:
#read the Operational_Data table from the Zip files for each year:
final_data = []

for i in range(len(e_Years)):
    if (i==len(e_Years)-1):
        dfc = pd.read_excel(io.BytesIO(read_zip(EIA20r + Current_year + '.zip',
                                        'Operational_Data_20' + Current_year + '.xlsx')), engine='openpyxl')
        dfc.columns = dfc.iloc[1]
        dfc =dfc.drop([0,1])
    else:
        df = pd.read_excel(io.BytesIO(read_zip(EIA20r + e_Years[i] + '.zip',
                                        'Operational_Data_20' + e_Years[i] + '.xlsx')), engine='openpyxl')
        df.columns = df.iloc[1]
        df =df.drop([0,1])
        final_data.append(df)
        
df = pd.concat(final_data)
df = df.append(dfc)
df = df.replace('.', '0') #replace missing values with character 0

col_list = ['Data Year', 'Utility Number', 'Summer Peak Demand', 'Winter Peak Demand', 'Net Generation', 'Wholesale Power Purchases','Exchange Energy Received', 'Exchange Energy Delivered', 'Net Power Exchanged', 'Wheeled Power Received', 'Wheeled Power Delivered', 'Net Wheeled Power', 'Transmission by Other Losses', 'Total Sources', 'Retail Sales', 'Sales for Resale','Furnished without Charge', 'Consumed by Respondent without Charge', 'Total Energy Losses', 'Total Disposition', 'From Retail Sales', 'From Delivery Customers', 'From Sales for Resale', 'From Credits or Adjustments', 'From Transmission', 'From Other','Total' ]
for col in col_list:
    df[col] = pd.to_numeric(df[col]) #convert colunn to float type

In [27]:
#create database file in directory path location or connect if exists:
conn = sqlite3.connect('eia_861.db')
cur = conn.cursor()

In [28]:
df.to_sql(name= "Operational_Data", con= conn, if_exists= 'replace', index = False)

C:\FERC_Apps\Anaconda\envs\new\lib\site-packages\pandas\core\generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [15]:
cur.execute("""
SELECT name 
FROM sqlite_master
WHERE type='table'
ORDER BY name; 
""")
print(cur.fetchall())

[('Operational_Data',)]


In [29]:
df = pd.read_sql_query("""
PRAGMA table_info(Operational_Data);
""", conn)
df

,cid,name,type,notnull,dflt_value,pk
0,0,Data Year,INTEGER,0,None,0
1,1,Utility Number,INTEGER,0,None,0
2,2,Utility Name,TEXT,0,None,0
3,3,State,TEXT,0,None,0
4,4,Ownership Type,TEXT,0,None,0
5,5,NERC Region,TEXT,0,None,0
6,6,Summer Peak Demand,REAL,0,None,0
7,7,Winter Peak Demand,REAL,0,None,0
8,8,Net Generation,INTEGER,0,None,0
9,9,Wholesale Power Purchases,INTEGER,0,None,0


In [17]:
df = pd.read_sql_query("""
SELECT "Data Year" as Year, "Utility Name" as Utility, "Sales for Resale" as Sales_for_Resale, "Ownership Type" as Ownership
FROM Operational_Data
WHERE Ownership in ('Cooperative', 'Federal','Municipal') and Sales_for_Resale > 0 and Year in (2018, 2019, 2020)
Order By Utility
""", conn)
df

,Year,Utility,Sales_for_Resale,Ownership
0,2018,Adams-Columbia Electric Coop,90,Cooperative
1,2019,Adams-Columbia Electric Coop,79,Cooperative
2,2020,Adams-Columbia Electric Coop,92,Cooperative
3,2018,Alaska Electric & Energy Coop,482977,Cooperative
4,2019,Alaska Electric & Energy Coop,477806,Cooperative
...,...,...,...,...
724,2019,Woodruff Electric Coop Corp,14523,Cooperative
725,2020,Woodruff Electric Coop Corp,13717,Cooperative
726,2018,Wyandotte Municipal Serv Comm,47038,Municipal
727,2019,Wyandotte Municipal Serv Comm,48683,Municipal


In [18]:
result = round(df.groupby(['Utility'])['Sales_for_Resale'].mean().reset_index())
result

,Utility,Sales_for_Resale
0,Adams-Columbia Electric Coop,87.0
1,Alaska Electric & Energy Coop,482315.0
2,"Allamakee-Clayton El Coop, Inc",9113.0
3,Allegheny Electric Coop Inc,3701057.0
4,Anchorage Municipal Light and Power,269514.0
...,...,...
280,"Western Farmers Elec Coop, Inc",12872976.0
281,"Wheatland Electric Coop, Inc",16128.0
282,Wolverine Power Supply Coop,7527924.0
283,Woodruff Electric Coop Corp,14504.0


In [19]:
#4 million----order 768?
de_minimus = 4000000

In [21]:
result = result.loc[(result['Sales_for_Resale'] > de_minimus)] #keep only sales for resale over "0"
result.reset_index()

,index,Utility,Sales_for_Resale
0,6,Arkansas Electric Coop Corp,16016984.0
1,10,Basin Electric Power Coop,30798813.0
2,12,Big Rivers Electric Corp,5798797.0
3,16,Bonneville Power Administration,82573272.0
4,17,Brazos Electric Power Coop Inc,17800408.0
5,19,"Buckeye Power, Inc",10204986.0
6,25,"Central Electric Pwr Coop, Inc - (SC)",18287420.0
7,106,City of San Antonio - (TX),6719460.0
8,131,Cooperative Energy,12285358.0
9,133,Dairyland Power Coop,6355960.0


In [ ]:
#result.to_csv('861_list_python_final.csv') #save in selected folder as csv